# Install dependencies


In [ ]:
!pip install -qU torch==2.7.1 torchvision==0.22.1 "ray[client,train]"==2.47.1

# Import dependencies


In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor

import ray
import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# Define functions


In [3]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"Train Epoch {epoch}")
        for X, y in train_dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            print(f"Test Epoch {epoch}")
            for X, y in test_dataloader:
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


@ray.remote
def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(
        num_workers=num_workers,
        use_gpu=use_gpu,
    )

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

# Connect to Ray Cluster


In [ ]:
ray.init(
    address="ray://raycluster-gpu-head-svc.default.svc.cluster.local:10001",
    runtime_env={"pip": ["torch==2.7.1", "torchvision==0.22.1"]},
)

2025-02-11 06:30:09,905	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.
2025-02-11 06:32:08,447	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.39.0
    Python: 3.9.20
This process on Ray Client was started with:
    Ray: 2.39.0
    Python: 3.9.13



Python version:,3.9.20
Ray version:,2.39.0
Dashboard:,http://10.0.3.222:8265


# Train model remotely


In [5]:
ray.get(train_fashion_mnist.remote(num_workers=1, use_gpu=True))

(autoscaler +7s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +7s) Adding 1 node(s) of type gpu-group.
(autoscaler +7s) Resized to 7 CPUs, 1 GPUs.
(train_fashion_mnist pid=355, ip=10.0.2.77) 
(train_fashion_mnist pid=355, ip=10.0.2.77) View detailed results here: /home/ray/ray_results/TorchTrainer_2025-02-10_22-36-46
(train_fashion_mnist pid=355, ip=10.0.2.77) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-02-10_21-50-27_650517_1/artifacts/2025-02-10_22-36-46/TorchTrainer_2025-02-10_22-36-46/driver_artifacts`
(train_fashion_mnist pid=355, ip=10.0.2.77) 
(train_fashion_mnist pid=355, ip=10.0.2.77) Training started with configuration:
(train_fashion_mnist pid=355, ip=10.0.2.77) ╭─────────────────────────────────────────────────╮
(train_fashion_mnist pid=355, ip=10.0.2.77) │ Training config                                 │
(train_fashion_mnist pid=355, ip=10.0.2.77) 

(RayTrainWorker pid=569, ip=10.0.2.77) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=491, ip=10.0.2.77) Started distributed worker processes: 
(TorchTrainer pid=491, ip=10.0.2.77) - (node_id=1275bcabf4e4c6be8a143540d0ba08d1b0229699b789f1c85f5708be, ip=10.0.2.77, pid=569) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=569, ip=10.0.2.77) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=569, ip=10.0.2.77) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0.00/26.4M [00:00<?, ?B/s] 
  0%|          | 65.5k/26.4M [00:00<01:12, 362kB/s]
  1%|          | 164k/26.4M [00:00<00:56, 466kB/s] 
  1%|          | 262k/26.4M [00:00<00:43, 602kB/s]
  2%|▏         | 426k/26.4M [00:00<00:33, 772kB/s]
  4%|▍         | 1.08M/26.4M [00:00<00:11, 2.24MB/s]
  8%|▊         | 2.23M/26.4M [00:00<00:05, 4.05MB/s]
 26%|██▋       | 6.98M/26.4M [00:01<00:01, 14.4MB/s]
 40%|███▉      | 10.5M/26.4M [00:01<00:00, 17.1MB/s]
 57%|█████▋    | 15.1M/26.4M [00:01<00:00, 24.1MB/s]
 72%|███████▏  | 18.9M/26.4M [00:01<00:00, 27.1MB/s]
 86%|████████▌ | 22.6M/26.4M [00:01<00:00, 25.8MB/s]
100%|██████████| 26.4M/26.4M [00:01<00:00, 16.0MB/s]


(RayTrainWorker pid=569, ip=10.0.2.77) Extracting /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=569, ip=10.0.2.77) 
(RayTrainWorker pid=569, ip=10.0.2.77) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
(RayTrainWorker pid=569, ip=10.0.2.77) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz


...

(RayTrainWorker pid=569, ip=10.0.2.77) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
(RayTrainWorker pid=569, ip=10.0.2.77) Extracting /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=569, ip=10.0.2.77) 


100%|██████████| 5.15k/5.15k [00:00<00:00, 66.8MB/s]


(RayTrainWorker pid=569, ip=10.0.2.77) Train Epoch 0


(RayTrainWorker pid=569, ip=10.0.2.77) Moving model to device: cuda:0


(RayTrainWorker pid=569, ip=10.0.2.77) Test Epoch 0
(train_fashion_mnist pid=355, ip=10.0.2.77) 
(train_fashion_mnist pid=355, ip=10.0.2.77) Training finished iteration 1 at 2025-02-10 22:37:09. Total running time: 22s
(train_fashion_mnist pid=355, ip=10.0.2.77) ╭───────────────────────────────╮
(train_fashion_mnist pid=355, ip=10.0.2.77) │ Training result               │
(train_fashion_mnist pid=355, ip=10.0.2.77) ├───────────────────────────────┤
(train_fashion_mnist pid=355, ip=10.0.2.77) │ checkpoint_dir_name           │
(train_fashion_mnist pid=355, ip=10.0.2.77) │ time_this_iter_s      19.1226 │
(train_fashion_mnist pid=355, ip=10.0.2.77) │ time_total_s          19.1226 │
(train_fashion_mnist pid=355, ip=10.0.2.77) │ training_iteration          1 │
(train_fashion_mnist pid=355, ip=10.0.2.77) │ accuracy               0.7958 │
(train_fashion_mnist pid=355, ip=10.0.2.77) │ loss                  0.54476 │
(train_fashion_mnist pid=355, ip=10.0.2.77) ╰───────────────────────────────╯
(

# Shut down Ray workers


In [6]:
ray.shutdown()

(train_fashion_mnist pid=355, ip=10.0.2.77) 
(train_fashion_mnist pid=355, ip=10.0.2.77) Training completed after 10 iterations at 2025-02-10 22:38:49. Total running time: 2min 2s
(train_fashion_mnist pid=355, ip=10.0.2.77) 
(train_fashion_mnist pid=355, ip=10.0.2.77) Training result: Result(
(train_fashion_mnist pid=355, ip=10.0.2.77)   metrics={'loss': 0.3545242327684983, 'accuracy': 0.873},
(train_fashion_mnist pid=355, ip=10.0.2.77)   path='/home/ray/ray_results/TorchTrainer_2025-02-10_22-36-46/TorchTrainer_90a23_00000_0_2025-02-10_22-36-46',
(train_fashion_mnist pid=355, ip=10.0.2.77)   filesystem='local',
(train_fashion_mnist pid=355, ip=10.0.2.77)   checkpoint=None
(train_fashion_mnist pid=355, ip=10.0.2.77) )


(train_fashion_mnist pid=355, ip=10.0.2.77) Wrote the latest version of all result files and experiment state to '/home/ray/ray_results/TorchTrainer_2025-02-10_22-36-46' in 0.0024s.
